In [3]:
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torch import nn

from nets.liteNet import FreezyNet

from glob import glob
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
import os
import pandas as pd


In [2]:
def average_non_zero_pixel(gray_image):
    non_zero_indices = np.nonzero(gray_image)
    non_zero_values = gray_image[non_zero_indices]
    average_value = np.mean(non_zero_values)
    return average_value

def get_green_yello_ratio(new,mask):
    hsv=cv2.cvtColor(new,cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 45, 0])
    upper_green = np.array([180, 255, 240])
    
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    new1 = cv2.bitwise_and(new,new,mask=green_mask)
    yellow_mask=255*mask-green_mask
    yellow_hsv=cv2.bitwise_and(hsv,hsv,mask=yellow_mask)
    green_pixels = cv2.countNonZero(green_mask)
    total_pixels = cv2.countNonZero(mask)
    h,s,v=cv2.split(hsv)
    h_value=average_non_zero_pixel(h)
    s_value=average_non_zero_pixel(s)
    h,s,v=cv2.split(yellow_hsv)
    h_y=average_non_zero_pixel(h)
    s_y=average_non_zero_pixel(s)
    if total_pixels==0:
        return 0
    ratio_green = green_pixels / total_pixels
    return ratio_green,green_pixels,total_pixels-green_pixels,h_value,s_value,h_y,s_y


In [3]:
table={}
for i in glob('../../phone_label/*/*'):
    fname=i.split('/')[-1].split('.')[0]
    table[fname]=i

input_shape=(512,512)
model = FreezyNet()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load('logs/FreezyNet_/phoneLite.pth', map_location=device))

model = nn.DataParallel(model)
model = model.cuda()
model = model.eval()

In [154]:
    
files=sorted(glob('VOCdevkit/VOC2007/SegmentationClass3/*C*.png'))
random.seed(6666)
random.shuffle(files)
files_s=files[:121]
validation=0.1
val_lines=files_s[:int(validation*len(files_s))]
train_lines=files_s[int(validation*len(files_s)):]
test_lines=files[121:]
len(files)


input_shape=(512,512)


In [6]:
input_shape=(512,512)
content='Plot,GreenLeafRation,YellowLeafRation,GreenLeaf,YellowLeafArea,TotalLeafArea,Hue,Sat\n'
save_path='phone_phenotypes.csv'
with open(save_path,'w',encoding='utf-8') as f:
    f.write(content)
load=[]   

files.sort()
with tqdm(total=len(files),desc='Progress:',postfix=dict,mininterval=0.3) as pbar:
    for index,file in enumerate(files):
        imgname=file.split('/')[-1].split('.')[0]
        image = cv2.imread(file)
        h,w=image.shape[:2]
        if h<w:
            image=cv2.rotate(image,cv2.ROTATE_90_CLOCKWISE)
            #label=cv2.rotate(label,cv2.ROTATE_90_CLOCKWISE)
            h,w=w,h
        ratio=512/w
        # find circle
        temp=image.copy()
        temp=cv2.resize(temp,(int(w*ratio),int(h*ratio)))
        gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
        b,g,r=cv2.split(temp)
        crt1=abs(gray.astype(np.int32)-r.astype(np.int32))<50
        crt2=abs(gray.astype(np.int32)-g.astype(np.int32))<50
        crt3=abs(gray.astype(np.int32)-b.astype(np.int32))<50
        rcrt1=r.astype(np.int32)-b.astype(np.int32)>15
        rcrt2=g.astype(np.int32)-b.astype(np.int32)>15
        rcrt3=abs(r.astype(np.int32)-g.astype(np.int32))>20
        rcrt4=cv2.bitwise_and(rcrt1.astype(np.uint8),rcrt2.astype(np.uint8))
        rcrt=cv2.bitwise_and(rcrt3.astype(np.uint8),rcrt4.astype(np.uint8))
        crt4=gray>100
        crt5=cv2.bitwise_and(crt1.astype(np.uint8),crt2.astype(np.uint8))
        crt6=cv2.bitwise_and(crt3.astype(np.uint8),crt5.astype(np.uint8))
        crt7=cv2.bitwise_and(crt6.astype(np.uint8),1-rcrt2.astype(np.uint8))
        mask=cv2.bitwise_and(crt4.astype(np.uint8),crt7.astype(np.uint8))
        temp = cv2.bitwise_and(temp, temp, mask=mask)
        gray = cv2.cvtColor(temp, cv2.COLOR_RGB2GRAY)
        kernel_size = (5, 5) 
        gray = cv2.blur(gray, kernel_size)
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 2, 500, param1=40, param2=80, minRadius=200, maxRadius=240)
        x1,x2,y1,y2=(round((circles[0][0][0]-circles[0][0][2])/ratio),
                 round((circles[0][0][0]+circles[0][0][2])/ratio),
                 round((circles[0][0][1]-circles[0][0][2])/ratio),
                 round((circles[0][0][1]+circles[0][0][2])/ratio))


        if y1<0:
            y1=0
        if y2>h:
            y2=h
        if x1<0:
            x1=0
        if x2>w:
            x2=w

        new=np.zeros((int(h),int(w),3),dtype=np.uint8)
        cv2.circle(new, (round(circles[0][0][0]/ratio), round(circles[0][0][1]/ratio)), round(circles[0][0][2]/ratio), (255, 255, 255), -1)
        
        image=cv2.bitwise_and(image,new)
        image=image[y1:y2,x1:x2,:]
        cv2.imwrite('{}'.format(os.path.basename(file)),image)
         #------------------------------------------#        
        image=cv2.resize(image,(input_shape[0],input_shape[0]))

        img=torch.from_numpy(image/255).float().cuda()
        img=img.unsqueeze(0)
        img=img.permute(0,3,1,2)

        with torch.no_grad():
            output=model(img)
        output = output[0].permute(1,2,0).cpu().argmax(axis=-1).numpy().astype(np.uint8)
        new = cv2.bitwise_and(image,image,mask=output)

        r,g,y,h,s,h_y,s_y=get_green_yello_ratio(new,output)  
        

        with open(save_path,'a',encoding='utf-8') as f:
            f.write('{},{},{},{},{},{},{},{},{},{}\n'.format(imgname,r,1-r,g,y,g+y,h,s,h_y,s_y))

        pbar.update(1)
        


Progress::   1%|▏         | 2/143 [00:02<02:34,  1.10s/it<class 'dict'>]
